In [1]:
#import data
import pandas as pd
from textblob import TextBlob
import pyrsm as rsm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
from sklearn.neural_network import MLPRegressor
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

interaction = pd.read_csv('RAW_interactions.csv')
recipe = pd.read_csv("RAW_recipes.csv")

* sentiment score

In [2]:
# drop rows with na
interaction = interaction.dropna()

# reset_index since we dropped some rows
interaction = interaction.reset_index()

# drop unneeded col
interaction = interaction.drop(columns=["index"])

# take first 100 rows for example
interaction_tmp = interaction.loc[0:99, :]

# add another col
interaction_tmp["score"] = 0

# use for loop to calcuate score
for i in range(0, interaction_tmp.shape[0]):
    interaction_tmp.loc[i, "score"] = TextBlob(interaction_tmp.loc[i, "review"]).sentiment[0]
    
# check result
interaction_tmp

<ipython-input-2-286de5f29a37>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interaction_tmp["score"] = 0
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,recipe_id,date,rating,review,score
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,0.270000
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",0.242857
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,0.366667
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,0.350000
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",-0.208333
...,...,...,...,...,...,...
95,456221,208980,2010-02-20,5,"This is a favorite here, too-- have made it se...",0.400000
96,229619,208980,2011-06-26,5,I was flipping through one of Rachael Ray's co...,-0.025000
97,280271,219118,2008-11-09,4,This is so yummy. Quick to prepare and can sim...,0.395578
98,305531,219118,2008-12-17,4,Really good carrots! There were a great side d...,0.625000


* decide how many data to use

In [3]:
# groupby recipe_id
interaction_groupby = interaction.groupby(by=["recipe_id"])["review"].count().sort_values(ascending=False)

# transform groupbyed result to dataframe
tmp = interaction_groupby.to_frame()
tmp = tmp.rename(columns={"review": "review_count"})

# see how many recipes are there
len(tmp.query("review_count > 80")) # 1067

# if > 80, assgin 1 in the selected col
tmp["selected"] = rsm.ifelse(tmp.review_count > 80, 1, 0)

# reset_index
tmp = tmp.reset_index()

# merge
interaction = interaction.merge(tmp, on="recipe_id", how="left") # left_join

# get selected data only
interaction_selected = interaction.query("selected == 1")

# reset_index
interaction_selected = interaction_selected.reset_index()

# drop unneeded col
interaction_selected = interaction_selected.drop(columns=["index"])

# add another col
interaction_selected["score"] = 0
interaction_selected

,user_id,recipe_id,date,rating,review,review_count,selected,score
0,2002084345,9054,2018-03-30,1,Mush,108,1,0
1,107135,135585,2006-04-02,5,i love buffulo wings and this makes it easy,103,1,0
2,53932,135585,2006-04-10,5,This is a lower calorie and neater way to enjo...,103,1,0
3,33159,135585,2006-04-23,5,Very satisfying & flavourful way to do chicken...,103,1,0
4,60260,135585,2006-04-26,5,Everyone loved this chicken! I used Frank's Ho...,103,1,0
...,...,...,...,...,...,...,...,...
189628,2001388634,38966,2017-02-12,1,NASTY,171,1,0
189629,2001452240,38966,2017-04-17,1,Question: I made this and I could taste the ba...,171,1,0
189630,2002038461,38966,2018-04-01,5,"Made this tonight, it’s so buttery and vanilla...",171,1,0
189631,2002134886,38966,2018-04-29,5,"My toddler loves to bake, and this recipe was ...",171,1,0


* applying TextBlob()

In [4]:
# use for loop to calcuate score
for i in range(0, interaction_selected.shape[0]):
    interaction_selected.loc[i, "score"] = TextBlob(interaction_selected.loc[i, "review"]).sentiment[0]

In [6]:
interaction_selected

,user_id,recipe_id,date,rating,review,review_count,selected,score
0,2002084345,9054,2018-03-30,1,Mush,108,1,0.000000
1,107135,135585,2006-04-02,5,i love buffulo wings and this makes it easy,103,1,0.466667
2,53932,135585,2006-04-10,5,This is a lower calorie and neater way to enjo...,103,1,0.175000
3,33159,135585,2006-04-23,5,Very satisfying & flavourful way to do chicken...,103,1,-0.037500
4,60260,135585,2006-04-26,5,Everyone loved this chicken! I used Frank's Ho...,103,1,0.063258
...,...,...,...,...,...,...,...,...
1996,808027,3470,2009-03-24,5,Great combination...great taste. Left out the...,322,1,0.033333
1997,229753,3470,2009-04-04,5,So delicious and easy.....I have made this 3 t...,322,1,0.272500
1998,1221921,3470,2009-04-04,5,This was delicious. I did make a couple change...,322,1,0.332612
1999,46080726,3470,2015-01-15,5,WOW!!! Delish!!!,322,1,0.381470


* calculate avg score

In [6]:
# groupby recipe_id
interaction_selected_groupby = interaction_selected.groupby(by=["recipe_id"])["score"].mean()
interaction_selected_groupby

# transform groupbyed result to dataframe
tmp = interaction_selected_groupby.to_frame()

# how many rows with score > 0.4
tmp.query("score > 0.4") # 121 rows

# reset_index
tmp = tmp.reset_index()
tmp

# change the name of recipe_id to id
tmp = tmp.rename(columns={"recipe_id": "id"})

# merge to get final dataset
df = tmp.merge(recipe, on="id", how="left") # left_join
df

,id,score,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,607,0.322297,famous barr s french onion soup,265,1634,1999-08-18,"['weeknight', 'time-to-make', 'course', 'main-...","[145.8, 10.0, 24.0, 45.0, 9.0, 19.0, 6.0]",18,"['peel onions and slice 1 / 8 inch thick , pre...",one of st. louis's favorite soup recipes was r...,"['onions', 'butter', 'black pepper', 'paprika'...",12
1,632,0.336054,100 whole wheat blueberry muffins,70,1530,1999-09-05,"['weeknight', 'time-to-make', 'course', 'main-...","[184.5, 10.0, 56.0, 5.0, 8.0, 5.0, 9.0]",7,"['preheat oven to 350', 'coat 12 muffin cups w...",NaN,"['whole wheat flour', 'baking soda', 'buttermi...",7
2,749,0.349448,zucchini lasagna lasagne low carb,60,1535,1999-09-14,"['60-minutes-or-less', 'time-to-make', 'course...","[261.1, 17.0, 42.0, 26.0, 48.0, 25.0, 5.0]",15,"['cook zucchini until tender , drain and set a...",great for anyone going low carb or who has so ...,"['zucchini', 'lean ground beef', 'onion', 'tom...",15
3,822,0.365344,heloise s cake mix cookies,8,1535,1999-08-09,"['15-minutes-or-less', 'time-to-make', 'course...","[137.4, 11.0, 46.0, 6.0, 2.0, 5.0, 5.0]",4,"['mix all ingredients well to remove lumps', '...",fast and easy when you don't have enough time ...,"['cake mix', 'vegetable oil', 'eggs']",3
4,860,0.336836,kentucky biscuits,23,1552,1999-08-26,"['30-minutes-or-less', 'time-to-make', 'course...","[160.7, 12.0, 5.0, 21.0, 5.0, 13.0, 6.0]",9,"['preheat oven to 425', 'sift into mixing bowl...",i got this recipe from a small free recipe boo...,"['flour', 'salt', 'sugar', 'baking powder', 'm...",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,486496,0.539946,bacon lattice tomato muffins rsc,60,2390546,2012-09-06,"['weeknight', '60-minutes-or-less', 'time-to-m...","[664.3, 71.0, 42.0, 68.0, 37.0, 101.0, 14.0]",20,['preheat oven to 400f line a rimmed baking sh...,"""ready, set, cook! contest entry."" bacon latti...","['reynolds wrap foil', 'bacon', 'flakey biscui...",11
1063,486641,0.420697,moist delicious banana nut bread,80,2009447,2012-09-09,"['30-minutes-or-less', 'time-to-make', 'course...","[177.8, 9.0, 65.0, 6.0, 5.0, 17.0, 9.0]",10,"['cream butter & sugar', 'add vanilla & eggs',...",i got this recipe when i was 12 and have used ...,"['butter', 'sugar', 'vanilla', 'eggs', 'banana...",10
1064,495271,0.519707,spinach and cheese stuffed chicken breast rsc,40,2689034,2013-02-11,"['weeknight', '60-minutes-or-less', 'time-to-m...","[409.4, 31.0, 6.0, 33.0, 82.0, 33.0, 5.0]",7,['butterfly chicken breast and drizzle with ol...,"ready, set, cook! hidden valley contest entry....","['boneless skinless chicken breasts', 'olive o...",9
1065,497021,0.520831,mile high cabbage pie 5fix,60,2390546,2013-03-09,"['weeknight', '60-minutes-or-less', 'time-to-m...","[803.0, 51.0, 42.0, 70.0, 97.0, 53.0, 25.0]",17,"['pre-heat oven to 350f', 'in stockpot , brown...",5-ingredient fix contest entry.\r\n have you ...,"['lean ground beef', 'coleslaw mix', 'simply p...",5


* get x and y for modeling

In [7]:
# specify X and y
X, y = df.loc[:, ["minutes", "n_steps", "n_ingredients"]], df["score"]

In [8]:
# splitting the train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, test_size=0.2, shuffle=False)

* MLP Regressor

In [11]:
# GridSearchCV
clf_rg = MLPRegressor(random_state=1234,
                      alpha=0.1,
                      hidden_layer_sizes=(2,2)).fit(X_train, y_train)

# CV for NN
nr_hnodes = range(1, 5)
hls = list(zip(nr_hnodes)) + list(zip(nr_hnodes, nr_hnodes))
hls

# alpha is the level of regularization
param_grid = {"hidden_layer_sizes": hls, "alpha": [0.001, 0.01, 0.05]}

clf_rg_cv = GridSearchCV(clf_rg, param_grid, scoring="neg_mean_squared_error", n_jobs=1, refit="RSME")
clf_rg_cv.fit(X_train, y_train)

clf_rg_cv.best_params_

{'alpha': 0.001, 'hidden_layer_sizes': (2, 2)}

In [12]:
# build mlp model using optimal parameters
clf_rg_cv = MLPRegressor(random_state=1234, 
                      hidden_layer_sizes=(2, 2), 
                      alpha = 0.001,
).fit(X_train, y_train)

In [13]:
# RMSE on testing

# pred
pred_test = clf_rg_cv.predict(X_test)

# RMSE
rmse_test = mean_squared_error(y_test, pred_test, squared = False)
rmse_test # 0.06938783589117235

0.06938783589117235

In [14]:
# RMSE on testing

# pred
pred_train = clf_rg_cv.predict(X_train)

# RMSE
rmse_test = mean_squared_error(y_train, pred_train, squared = False)
rmse_test # 0.051123341016958986

0.051123341016958986

* XGB Regressor

In [17]:
# GridSearchCV
xgb_reg_param_grid = {
    'n_estimators': range(10,100,10),
    'max_depth': range(2, 12)
}

# Instantiate the regressor: gbm
xgb_reg = xgb.XGBRegressor(n_estimators=10)

# Perform random search and fit to the data
xgb_reg_cv = GridSearchCV(xgb_reg, xgb_reg_param_grid,
                                scoring="neg_mean_squared_error",refit='RSME',
                                cv=5,verbose=1).fit(X_train,y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


In [19]:
print("Best parameters found: ", xgb_reg_cv.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(xgb_reg_cv.best_score_)))

Best parameters found:  {'max_depth': 2, 'n_estimators': 10}
Lowest RMSE found:  0.04982786208191918


In [20]:
# build model and apply best parameters
xgb_reg_good = xgb.XGBRegressor(
    max_depth=4,
    n_estimators=10,
    random_state=1234
).fit(X_train, y_train, verbose=True)

In [21]:
# RMSE on testing

# pred
pred = xgb_reg_good.predict(X_test)

# RMSE
rmse = np.sqrt(mean_squared_error(y_test,pred))
rmse # 0.062478621866004544

0.062478621866004544

In [22]:
# RMSE on training

# pred
pred = xgb_reg_good.predict(X_train)

# RMSE
rmse = np.sqrt(mean_squared_error(y_train, pred))
rmse # 0.04468574263366591

0.04468574263366591

Choose xgb since its RMSE is smaller.